# Used libraries

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import collections
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

## Loading data

The train and test inputs are composed of 46 features.

The target of this challenge is `RET` and corresponds to the fact that the **return is in the top 50% of highest stock returns**.

Since the median is very close to 0, this information should not change much with the idea to predict the sign of the return.

In [2]:
x_train = pd.read_csv('../x_train.csv', index_col='ID')
y_train = pd.read_csv('../y_train.csv', index_col='ID')
train = pd.concat([x_train, y_train], axis=1)
test = pd.read_csv('../x_test.csv', index_col='ID')
train.head()

,DATE,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,VOLUME_1,RET_2,VOLUME_2,...,VOLUME_16,RET_17,VOLUME_17,RET_18,VOLUME_18,RET_19,VOLUME_19,RET_20,VOLUME_20,RET
ID,,,,,,,,,,,,,,,,,,,,,
0,0,2,18,5,3,44,-0.015748,0.147931,-0.015504,0.179183,...,0.630899,0.003254,-0.379412,0.008752,-0.110597,-0.012959,0.174521,-0.002155,-0.000937,True
1,0,3,43,15,6,104,0.003984,NaN,-0.090580,NaN,...,NaN,0.003774,NaN,-0.018518,NaN,-0.028777,NaN,-0.034722,NaN,True
2,0,4,57,20,8,142,0.000440,-0.096282,-0.058896,0.084771,...,-0.010336,-0.017612,-0.354333,-0.006562,-0.519391,-0.012101,-0.356157,-0.006867,-0.308868,False
3,0,8,1,1,1,2,0.031298,-0.429540,0.007756,-0.089919,...,0.012105,0.033824,-0.290178,-0.001468,-0.663834,-0.013520,-0.562126,-0.036745,-0.631458,False
4,0,14,36,12,5,92,0.027273,-0.847155,-0.039302,-0.943033,...,-0.277083,-0.012659,0.139086,0.004237,-0.017547,0.004256,0.579510,-0.040817,0.802806,False


In [3]:
ret=[f'RET_{k}' for k in range(20,0,-1)]
vol=[f'VOLUME_{k}' for k in range(20,0,-1)]
features = ret+vol
target = 'RET'

# Cross-validation

In [17]:
K = 5 # Nombre de folds
KF = KFold(K,shuffle=True,random_state=1)

In [18]:
dates = train.groupby('DATE').count().reset_index()['DATE']

In [19]:
seuil_reel = [999999,999999,999999,999999,20000] #Seuils réels en cas de submit, ne pas réduire selon le nombre de folds
nb_shifts = 5 #nombre de features supplémentaires
seuil_ret = 1000
seuil_vol = 1000

In [20]:
categories = ['STOCK','SUB_INDUSTRY','INDUSTRY','INDUSTRY_GROUP','SECTOR']
features_base = ret+vol

In [21]:
scores = []
seuil = seuil_reel*(1-1/K)
for fold,(ind_date_train,ind_date_test) in enumerate(KF.split(dates)):
    # Extraction d'un train set et test set d'après les dates
    train_cl = train.loc[train['DATE'].isin(dates[ind_date_train].values)]
    test_cl = train.loc[train['DATE'].isin(dates[ind_date_test].values)]

    # Nettoyage
    for ind_groupe in np.intersect1d(train['INDUSTRY_GROUP'].unique(),test['INDUSTRY_GROUP'].unique()):
        for ret_t,vol_t in zip(ret,vol):
                med_ret = train_cl[ret_t][train['INDUSTRY_GROUP']==ind_groupe].median()
                med_vol = train_cl[vol_t][train['INDUSTRY_GROUP']==ind_groupe].median()

                train_cl.loc[:,ret_t] = train_cl.loc[:,ret_t].fillna(med_ret)
                train_cl.loc[:,vol_t] = train_cl.loc[:,vol_t].fillna(med_vol)
                test_cl.loc[:,ret_t] = test_cl.loc[:,ret_t].fillna(med_ret)
                test_cl.loc[:,vol_t] = test_cl.loc[:,vol_t].fillna(med_vol)
            
    train_cl  = train_cl[np.all([train_cl[col]<seuil_ret for col in ret],axis=0)]
    train_cl  = train_cl[np.all([train_cl[col]<seuil_vol for col in vol],axis=0)]
            
    scaler = StandardScaler()
    train_cl.loc[:,ret+vol] = scaler.fit_transform(train_cl.loc[:,ret+vol])
    test_cl.loc[:,ret+vol] = scaler.fit_transform(test_cl.loc[:,ret+vol])

    # Sélection des groupes à conserver
    model_labels = [[] for ind_cat in range(len(categories))]
    for ind_cat,cat in enumerate(categories):
        count_group = pd.Series(collections.Counter(train_cl[cat])).sort_index()
        
        for group in count_group.index:
            if count_group.loc[group]>=seuil[ind_cat]:
                model_labels[ind_cat].append(group)
        print(f"{len(model_labels[ind_cat])} modèles spécifiques de {cat} vont être entraînés")

    # New features créées par clusters + entraînement des modèles

    models = [{} for ind_cat in range(len(categories)+1)]
    
    for ind_cat,cat in enumerate(categories):

        for ind_group,group in enumerate(model_labels[ind_cat]):
            x_train_cl = train_cl.copy()[train_cl[cat]==group]
            y_train_cl = train_cl[target][train_cl[cat]==group]
            
#             ind_cat_newfeat = ind_cat-(ind_cat>0)
            ind_cat_new_feat = ind_cat
            new_features = []
            shifts = range(1,nb_shifts+1)
            statistics = ['median']
            gb_features = [categories[ind_cat_newfeat],'DATE']
            target_feature = 'RET'
            tmp_name = '_'.join(gb_features)
            for shift in shifts:
                for stat in statistics:
                    name = f'{target_feature}_{shift}_{tmp_name}_{stat}'
                    feat = f'{target_feature}_{shift}'
                    new_features.append(name)
                    x_train_cl[name] = x_train_cl.groupby(gb_features)[feat].transform(stat)

            features = features_base + new_features

            rf_params = {
            'n_estimators': 500,
            'max_depth': 2**3,
            'random_state': 0,
            'n_jobs': -1
            }

            model = RandomForestClassifier(**rf_params)
            model.fit(x_train_cl[features], y_train_cl)
            models[ind_cat][group] = model
            print(f"Modèle {cat} {group} entraîné : {len(x_train_cl)} données")

    #On entraîne un modèle général pour les données restantes
    x_train_cl = train_cl.copy()
    y_train_cl = train_cl[target]

    new_features = []
    shifts = range(1,nb_shifts+1)
    statistics = ['median']
#     gb_features = ['SECTOR','DATE']
    gb_features = ['DATE']
    target_feature = 'RET'
    tmp_name = '_'.join(gb_features)
    for shift in shifts:
        for stat in statistics:
            name = f'{target_feature}_{shift}_{tmp_name}_{stat}'
            feat = f'{target_feature}_{shift}'
            new_features.append(name)
            x_train_cl[name] = x_train_cl.groupby(gb_features)[feat].transform(stat)

    features = features_base + new_features

    rf_params = {
    'n_estimators': 500,
    'max_depth': 2**3,
    'random_state': 0,
    'n_jobs': -1
    }

    model = RandomForestClassifier(**rf_params)
    model.fit(x_train_cl[features], y_train_cl)
    models[5]['general'] = model
    print(f"Modèle général entraîné : {len(x_train_cl)} données")

    # Prédictions
    test_temp = test_cl.copy()
    y_pred = pd.Series()
    
    for ind_cat,cat in enumerate(categories):
        for ind_group,group in enumerate(model_labels[ind_cat]):
            x_test = test_temp.loc[test_temp[cat]==group,:]

#             ind_cat_newfeat = ind_cat-(ind_cat>0)
            ind_cat_new_feat = ind_cat
            new_features = []
            shifts = range(1,nb_shifts+1)
            statistics = ['median']
            gb_features = [categories[ind_cat_newfeat],'DATE']
            target_feature = 'RET'
            tmp_name = '_'.join(gb_features)
            for shift in shifts:
                for stat in statistics:
                    name = f'{target_feature}_{shift}_{tmp_name}_{stat}'
                    feat = f'{target_feature}_{shift}'
                    new_features.append(name)
                    x_test[name] = x_test.groupby(gb_features)[feat].transform(stat)

            features = features_base + new_features                

            index = x_test.index
            model = models[ind_cat][group]
            y_pred = pd.concat([y_pred,pd.Series(model.predict(x_test[features]),index=x_test.index)])
            print(f"{len(x_test)} données prédites avec le modèle {cat} {group}")

            test_temp = test_temp[test_temp[cat]!=group] #On vire les données utilisées

    #Il ne reste que les données à traiter avec le modèle général
    if len(test_temp)>0:
        x_test = test_temp.copy()

        new_features = []
        shifts = range(1,nb_shifts+1)
        statistics = ['median']
#         gb_features = ['SECTOR','DATE']
        gb_features = ['DATE']
        target_feature = 'RET'
        tmp_name = '_'.join(gb_features)
        for shift in shifts:
            for stat in statistics:
                name = f'{target_feature}_{shift}_{tmp_name}_{stat}'
                feat = f'{target_feature}_{shift}'
                new_features.append(name)
                x_test[name] = x_test.groupby(gb_features)[feat].transform(stat)

        features = features_base + new_features   

        index = x_test.index
        model = models[5]['general']
        y_pred = pd.concat([y_pred,pd.Series(model.predict(x_test[features]),index=x_test.index)])
        print(f"{len(x_test)} données prédites avec le modèle général")

    y_pred = y_pred[test_cl.index]

    score = accuracy_score(test_cl[target], y_pred)
    print(f"Score fold {fold+1} : {score* 100:.2f}")
    scores.append(score)

# Résultats finaux :
mean = np.mean(scores)*100
std = np.std(scores)*100
u = (mean + std)
l = (mean - std)
print(f'Accuracy : {mean:.2f}% [{l:.2f} ; {u:.2f}] (+- {std:.2f})')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0 modèles spécifiques de STOCK vont être entraînés
0 modèles spécifiques de SUB_INDUSTRY vont être entraînés
0 modèles spécifiques de INDUSTRY vont être entraînés
0 modèles spécifiques de INDUSTRY_GROUP vont être entraînés
6 modèles spécifiques de SECTOR vont être entraînés
Modèle SECTOR 1 entraîné : 16930 données
Modèle SECTOR 3 entraîné : 44155 données
Modèle SECTOR 4 entraîné : 50441 données
Modèle SECTOR 6 entraîné : 43949 données
Modèle SECTOR 7 entraîné : 69608 données
Modèle SECTOR 8 entraîné : 56212 données
Modèle général entraîné : 332932 données


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


4334 données prédites avec le modèle SECTOR 1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

11215 données prédites avec le modèle SECTOR 6


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


17267 données prédites avec le modèle SECTOR 7


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


14039 données prédites avec le modèle SECTOR 8
13279 données prédites avec le modèle général
Score fold 5 : 50.89
Accuracy : 51.18% [50.63 ; 51.74] (+- 0.55)
